In IF NBCn1, multiple hydrophobic and polar residues on TM3 and the GATE domain occlude the passage of water from the extra cellular side.

Similarly multiple residues between TM10 and GATE, occlude the passage from the intra cellular side in OF NBCn1.

Here we plot the free energy in terms of distances between TM3, TM10 and GATE. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pyemma
# for visualization of molecular structures:
import pickle
import mdtraj
import glob
import os
from threading import Timer

In [ ]:
TMs = ["TM1", "TM2", "TM3", "TM4", "TM8", "TM9", "TM10", "TM11"] # TM helices of Core domain
list_of_trajs = []
start = 1

nfolders = np.arange(1,17)

"""
There are two sets of simulations - sim1 and sim2. The *.xvg files in the data folders contain average distances between
center of mass (COM) of TMs in the CORE domain and COM  of the GATE domain,
The files were generated by using gromacs tool gmx distance using the following command
gmx_mpi distance -f traj.xtc -s ../folder1/ref.tpr -n ../index.ndx -oav distav_TM1_1us.xvg -select 'cog of group GATE plus cog of group TM1'
GATE and TM1 are defined as group of atoms in the index file index.ndx
"""
for nfolder in nfolders:
    folder = "/path_to_data/nbcn1_eq_sim1/folder" + str(nfolder)
    for TM in TMs:
        feature_file = os.path.join(folder , "distav_" + TM + "_1us.xvg")
        raw_data = np.loadtxt(feature_file, comments = ["#", "@"])
        
        if(TM==TMs[0]):
            feature_array = np.array(raw_data[start:,1]*10.0)
        else:
            feature_array = np.column_stack((feature_array, np.array(raw_data[start:,1])*10.0))
    list_of_trajs.append(feature_array)

for nfolder in nfolders:
    folder = "/path_to_data/nbcn1_eq_sim2/folder" + str(nfolder)
    for TM in TMs:
        feature_file = os.path.join( folder , "distav_" + TM + "_1us.xvg")
        raw_data = np.loadtxt(feature_file, comments = ["#", "@"])
        
        if(TM==TMs[0]):
            feature_array = np.array(raw_data[start:,1]*10.0)
        else:
            feature_array = np.column_stack((feature_array, np.array(raw_data[start:,1])*10.0))
    list_of_trajs.append(feature_array)

In [ ]:
# Concatenating the data for plotting
data_conc = np.concatenate(list_of_trajs, axis=0)
np.shape(data_conc)

In [ ]:
# We cluster the cumulative data into 1200 clusters
cluster = pyemma.coordinates.cluster_kmeans(list_of_trajs, k=1200, max_iter=1000, stride=10, n_jobs=8)
dtrajs = cluster.dtrajs
dtrajs_conc = np.concatenate(dtrajs)

In [ ]:
# Reading in the trajectory files - Using source to reduce memory requirements
# the combined xtc file contains all the trajectories and is generated using gmx trjcat
reader_run1 = pyemma.coordinates.source("path_to_data/nbcn1_comb_1us_dt_0.1ns.xtc", top="path_to_data/nbcn1_eq_sim1/folder1/input.gro") # create reader

In [ ]:
# Free energy profile to observe stationary states
# index 2 corresponds to TM3 distance and 6 corresponds to TM10 distance
fig, axes = pyemma.plots.plot_free_energy(data_conc[:,2], 
                                          data_conc[:,6], nbins=50, 
                                          kT=0.616, cbar_label='free energy / kcal/mol')
axes.set_xlabel('Extracellular distance (${Å}$)', fontsize=16)
axes.set_ylabel('Intracellular distance (${Å}$)', fontsize=16)

In [ ]:
# Positions of the metastable states corresponding to IF, Int and OF states from the graph generated in the cell above
# The first coordinate is the TM3 distance and second component is the TM10 distance
IF = [19.1, 20.7]
Int = [21.8, 16.5]
OF = [28.6, 12.6]


In [ ]:
# Initializing a dictionary to store nearest cluster information
dict_states = {}

In [ ]:
import numpy as np

def find_closest_values(data, col1, val1, col2, val2):
    # Calculate the difference between the target values and the data
    diff1 = np.abs(data[:, col1] - val1)
    diff2 = np.abs(data[:, col2] - val2)
    
    # Sum the differences to get a combined distance
    combined_diff = diff1 + diff2
    
    # Find the index of the row with the smallest combined difference
    closest_index = np.argmin(combined_diff)
    
    # Get the values from the closest row
    closest_row = data[closest_index]
    
    # Get the closest values from the specified columns
    closest_val1 = closest_row[col1]
    closest_val2 = closest_row[col2]
    
    return closest_index, closest_val1, closest_val2, closest_row

# Columns corresponding to TM3 (index 2) and TM10 (index 6) distances
col1, val1 = 2, OF[0]  
col2, val2 = 6, OF[1]

index, closest_val1, closest_val2, closest_row = find_closest_values(data_conc, col1, val1, col2, val2)
print(f"Index: {index}")
print(f"Closest value in column {col1}: {closest_val1}")
print(f"Closest value in column {col2}: {closest_val2}")
print(f"Corresponding row values: {closest_row}")
dict_states["OF"] = closest_row

col1, val1 = 2, IF[0]  
col2, val2 = 6, IF[1]

index, closest_val1, closest_val2, closest_row = find_closest_values(data_conc, col1, val1, col2, val2)
print(f"Index: {index}")
print(f"Closest value in column {col1}: {closest_val1}")
print(f"Closest value in column {col2}: {closest_val2}")
print(f"Corresponding row values: {closest_row}")
dict_states["IF"] = closest_row

col1, val1 = 2, Int[0]  
col2, val2 = 6, Int[1]

index, closest_val1, closest_val2, closest_row = find_closest_values(data_conc, col1, val1, col2, val2)
print(f"Index: {index}")
print(f"Closest value in column {col1}: {closest_val1}")
print(f"Closest value in column {col2}: {closest_val2}")
print(f"Corresponding row values: {closest_row}")
dict_states["Int"] = closest_row

In [ ]:
dict_states #Verify if the information is captured accurately

In [ ]:
# Plotting free energy with IF, Int and OF identified
plt.figure(figsize = (8, 8), dpi = 600)
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams.update({'figure.autolayout': True})
plt.rcParams['lines.markersize'] = 16



fig, axes = pyemma.plots.plot_free_energy(data_conc[:,2], 
                                          data_conc[:,6], nbins=50, 
                                          kT=0.616, cbar_label='free energy / kcal/mol')
axes.set_xlabel('Extracellular distance (${Å}$)', fontsize=16)
axes.set_ylabel('Intracellular distance (${Å}$)', fontsize=16)

count = 1
for state in dict_states:
    point = np.array(dict_states[state])
    nearest_point = find_nearest_point(point, cluster_tica.clustercenters)
    print(state, nearest_point)
    cl= nearest_point[1]
    color = "white"
    label = state
    axes.scatter(cluster_tica.clustercenters[cl,2], cluster_tica.clustercenters[cl,6], color = color, marker="*")
    axes.annotate(label, (cluster_tica.clustercenters[cl,2], cluster_tica.clustercenters[cl,6]),xytext=(0, 10), textcoords='offset points', ha='center',  fontsize=14, color=color)
fig.savefig("/path_to_results/Figure_name.png", dpi=600) 


In [ ]:
# Extracting xtc files for water density computation and representative PDB structure from the nearest cluster.
for state in dict_states:
    point = np.array(dict_states[state])
    nearest_point = find_nearest_point(point, cluster_tica.clustercenters)
    print(state, nearest_point)
    cl= nearest_point[1]
    microstate_samples = cluster_tica.sample_indexes_by_cluster([cl], nsample=500, replace=False)[0]
    unique_sampl, inverse_sampl = np.unique(microstate_samples[:,1], return_index=True)
    pyemma.coordinates.save_traj(reader_run1, microstate_samples[inverse_sampl], '/path_to_results/cluster_traj_' + state + '_str.xtc')
    microstate_samples = cluster_tica.sample_indexes_by_cluster([cl], nsample=1, replace=False)[0]
    unique_sampl, inverse_sampl = np.unique(microstate_samples[:,1], return_index=True)
    pyemma.coordinates.save_traj(reader_run1, microstate_samples[inverse_sampl], '/path_to_results/cluster_sample_' + state + '_str.pdb')